In [4]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import LabelEncoder

In [5]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [6]:
pizza_df = pd.read_csv('pizza_sales.csv')
ingred_df = pd.read_csv('Pizza_ingredients.csv')

In [7]:
pizza_df.dropna(inplace=True)

In [8]:
def parse_dates(date):
  for fmt in ('%d-%m-%Y', '%d/%m/%Y'):
    try:
      return pd.to_datetime(date, format=fmt)
    except ValueError:
      pass
  raise ValueError(f'no valid date format found for {date}')

In [9]:
pizza_df['order_date'] = pizza_df['order_date'].apply(parse_dates)

In [10]:
pizza_df = pizza_df[['order_date', 'pizza_name', 'quantity']]

In [11]:
# pizza_df['pizza_name'].unique()

In [12]:
sales_summary = pizza_df.groupby(['order_date', 'pizza_name']).sum().reset_index()

In [13]:
sales_summary.head()

,order_date,pizza_name,quantity
0,2015-01-01,The Barbecue Chicken Pizza,11
1,2015-01-01,The Big Meat Pizza,5
2,2015-01-01,The Calabrese Pizza,1
3,2015-01-01,The California Chicken Pizza,4
4,2015-01-01,The Chicken Alfredo Pizza,2


In [14]:
# reshape data for time series modeling
sales_pivot = sales_summary.pivot(index='order_date', columns='pizza_name', values='quantity').fillna(0)

In [15]:
arima_models = {}

for pizza_name in sales_pivot.columns:
  try:
    model = ARIMA(sales_pivot[pizza_name], order=(1, 1, 0))
    model_fit = model.fit()
    arima_models[pizza_name] = model_fit
  except:
    print(f'ARIMA model for {pizza_name} failed to fit')

In [16]:
# Generate predictions for one week
prediction_days = 7
predictions_arima = {}

for pizza_name, model in arima_models.items():
  predictions_arima[pizza_name] = model.predict(start=len(sales_pivot), end=len(sales_pivot) + prediction_days - 1)

In [18]:
predictions_df = pd.DataFrame(predictions_arima)

In [21]:
predictions_df.index = pd.date_range(start=sales_pivot.index[-1], periods=prediction_days, freq='D')

In [26]:
ingredients_df = ingred_df[['pizza_name', 'pizza_ingredients', 'Items_Qty_In_Grams']]

In [27]:
# change column name to items_qty
ingredients_df.rename(columns={'Items_Qty_In_Grams': 'items_qty'}, inplace=True)

In [28]:
predictions_df

,The Barbecue Chicken Pizza,The Big Meat Pizza,The Brie Carre Pizza,The Calabrese Pizza,The California Chicken Pizza,The Chicken Alfredo Pizza,The Chicken Pesto Pizza,The Classic Deluxe Pizza,The Five Cheese Pizza,The Four Cheese Pizza,...,The Prosciutto and Arugula Pizza,The Sicilian Pizza,The Soppressata Pizza,The Southwest Chicken Pizza,The Spicy Italian Pizza,The Spinach Pesto Pizza,The Spinach Supreme Pizza,The Spinach and Feta Pizza,The Thai Chicken Pizza,The Vegetables + Vegetables Pizza
2015-12-31,7.949231,4.420315,0.868806,1.368239,7.0,2.515974,2.139860,6.884102,3.032327,6.974960,...,3.594000,2.583928,2.060907,5.917019,5.844288,3.542421,2.0,3.173030,3.0,7.773728
2016-01-01,9.000644,5.252117,1.508606,2.255787,7.0,3.250085,3.004890,8.502239,4.000261,8.000157,...,4.252945,3.751409,2.501855,5.496557,7.266934,4.250600,2.0,4.007485,3.0,8.764985
2016-01-02,8.461593,4.814122,1.146737,1.773032,7.0,2.886938,2.602621,7.661914,3.524117,7.481141,...,3.944120,3.187265,2.294809,5.689343,6.625582,3.906524,2.0,3.626354,3.0,8.323624
2016-01-03,8.737960,5.044753,1.351409,2.035612,7.0,3.066578,2.789690,8.098308,3.758341,7.743898,...,4.088856,3.459867,2.392027,5.600949,6.914714,4.073697,2.0,3.800433,3.0,8.520142
2016-01-04,8.596269,4.923311,1.235647,1.892789,7.0,2.977715,2.702696,7.871682,3.643122,7.610875,...,4.021023,3.328142,2.346379,5.641478,6.784369,3.992474,2.0,3.720923,3.0,8.432641
2016-01-05,8.668913,4.987258,1.301122,1.970474,7.0,3.021673,2.743152,7.989373,3.699800,7.678219,...,4.052814,3.391794,2.367813,5.622895,6.843130,4.031937,2.0,3.757239,3.0,8.471601
2016-01-06,8.631669,4.953586,1.264089,1.928219,7.0,2.999928,2.724338,7.928254,3.671919,7.644125,...,4.037915,3.361036,2.357748,5.631416,6.816640,4.012764,2.0,3.740652,3.0,8.454254


In [29]:
ingredients_df.head()

,pizza_name,pizza_ingredients,items_qty
0,The Barbecue Chicken Pizza,Barbecued Chicken,40.0
1,The Barbecue Chicken Pizza,Red Peppers,15.0
2,The Barbecue Chicken Pizza,Green Peppers,20.0
3,The Barbecue Chicken Pizza,Tomatoes,30.0
4,The Barbecue Chicken Pizza,Red Onions,60.0


In [30]:

# Create a dictionary to store the ingredient quantities
ingredient_quantities = {}

# Iterate through each pizza in the predictions
for pizza_name in predictions_df.columns:
  # Get the predicted quantity for the pizza
  predicted_quantity = predictions_df[pizza_name].sum()

  # Get the ingredients for the pizza
  pizza_ingredients = ingredients_df[ingredients_df['pizza_name'] == pizza_name]

  # Iterate through each ingredient for the pizza
  for index, row in pizza_ingredients.iterrows():
    ingredient = row['pizza_ingredients']
    ingredient_qty = row['items_qty']

    # Calculate the required quantity of the ingredient
    required_quantity = predicted_quantity * ingredient_qty

    # Add the required quantity to the dictionary
    if ingredient not in ingredient_quantities:
      ingredient_quantities[ingredient] = 0
    ingredient_quantities[ingredient] += required_quantity




In [31]:
# Create a DataFrame from the ingredient quantities
ingredient_requirements_df = pd.DataFrame.from_dict(ingredient_quantities, orient='index', columns=['required_quantity'])



                         required_quantity
Barbecued Chicken              5404.165210
Red Peppers                   11341.551998
Green Peppers                  8030.393870
Tomatoes                      34984.718341
Red Onions                    54797.556512
...                                    ...
Luganega Sausage               2767.612913
Onions                         1383.806456
Soppressata Salami             4896.461505
Peperoncini verdi              1412.869703
Thai Sweet Chilli Sauce        1260.000000

[64 rows x 1 columns]


In [32]:
# Print the ingredient requirements
ingredient_requirements_df

,required_quantity
Barbecued Chicken,5404.165210
Red Peppers,11341.551998
Green Peppers,8030.393870
Tomatoes,34984.718341
Red Onions,54797.556512
...,...
Luganega Sausage,2767.612913
Onions,1383.806456
Soppressata Salami,4896.461505
Peperoncini verdi,1412.869703


In [33]:


# Create a purchase order DataFrame
purchase_order_df = ingredient_requirements_df.copy()

# Add a column for the unit of measure (assuming all ingredients are in grams)
purchase_order_df['unit'] = 'grams'

# Rename the columns for better readability
purchase_order_df = purchase_order_df.rename(columns={'required_quantity': 'quantity'})

# Add a header to the purchase order
print('Purchase Order:')
print('----------------')

# Print the purchase order table
print(purchase_order_df.to_string())


Purchase Order:
----------------
                                quantity   unit
Barbecued Chicken            5404.165210  grams
Red Peppers                 11341.551998  grams
Green Peppers                8030.393870  grams
Tomatoes                    34984.718341  grams
Red Onions                  54797.556512  grams
Barbecue Sauce               1801.388403  grams
Bacon                       19992.004764  grams
Pepperoni                   24192.916429  grams
Italian Sausage               343.954622  grams
Chorizo Sausage              1719.773109  grams
Brie Carre Cheese             260.292444  grams
Prosciutto                    260.292444  grams
Caramelized Onions                   NaN  grams
Pears                          86.764148  grams
Thyme                          43.382074  grams
Garlic                      17939.075392  grams
?duja Salami                 1586.898271  grams
Pancetta                     2380.347406  grams
Friggitello Peppers           396.724568  grams
Chicken